In [1]:
import sqlite3
import pandas as pd
import numpy as np
import sys
import glob, os
from pyteomics import mgf
from os.path import expanduser

In [2]:
BASE_MAXQUANT_DIR = '{}/MQ-analysis-of-P3856/combined'.format(expanduser('~'))
MAXQUANT_TXT_DIR = '{}/txt'.format(BASE_MAXQUANT_DIR)
ALLPEPTIDES_FILENAME = '{}/allPeptides.txt'.format(MAXQUANT_TXT_DIR)
APL_DIR = '{}/andromeda'.format(BASE_MAXQUANT_DIR)

In [3]:
RT_LOWER = 1650
RT_UPPER = 2200

In [4]:
run_name = 'P3856_YHE211_1_Slot1-1_1_5104'

In [5]:
allpeptides_df = pd.read_csv(ALLPEPTIDES_FILENAME, sep='\t')
allpeptides_df.rename(columns={'Number of isotopic peaks':'isotope_count', 'm/z':'mz', 'Number of data points':'number_data_points', 'Intensity':'intensity', 'Ion mobility index':'scan', 'Ion mobility index length':'scan_length', 'Ion mobility index length (FWHM)':'scan_length_fwhm', 'Retention time':'rt', 'Retention length':'rt_length', 'Retention length (FWHM)':'rt_length_fwhm', 'Charge':'charge_state', 'Number of pasef MS/MS':'number_pasef_ms2_ids', 'Pasef MS/MS IDs':'pasef_msms_ids', 'MS/MS scan number':'msms_scan_number', 'Isotope correlation':'isotope_correlation'}, inplace=True)
# allpeptides_df = allpeptides_df[allpeptides_df.intensity.notnull() & (allpeptides_df.number_pasef_ms2_ids > 0) & (allpeptides_df.msms_scan_number >= 0) & allpeptides_df.pasef_msms_ids.notnull()].copy()
allpeptides_df = allpeptides_df[(allpeptides_df.msms_scan_number >= 0)].copy()
allpeptides_df['rt_in_seconds'] = allpeptides_df.rt * 60.0
allpeptides_df.sort_values(by=['msms_scan_number'], ascending=True, inplace=True)

In [6]:
allpeptides_df.msms_scan_number = allpeptides_df.msms_scan_number.apply(lambda x: int(x))

In [7]:
allpeptides_df.msms_scan_number.max()

25266

In [8]:
allpeptides_df.columns

Index(['Raw file', 'charge_state', 'mz', 'Mass', 'number_data_points',
       'Number of frames', 'isotope_count', 'isotope_correlation',
       'Mass fractional part', 'Mass deficit', 'rt', 'rt_length',
       'rt_length_fwhm', 'Min frame index', 'Max frame index', 'scan',
       'scan_length', 'scan_length_fwhm', 'intensity', 'Intensities',
       'number_pasef_ms2_ids', 'pasef_msms_ids', 'msms_scan_number',
       'rt_in_seconds'],
      dtype='object')

In [9]:
allpeptides_df['Raw file'].unique()

array(['P3856_YHE010_8_Slot1-1_1_5100', 'P3856_YHE114_10_Slot1-1_1_5124',
       'P3856_YHE211_9_Slot1-1_1_5112', 'P3856_YHE010_6_Slot1-1_1_5098',
       'P3856_YHE211_10_Slot1-1_1_5113', 'P3856_YHE114_3_Slot1-1_1_5117',
       'P3856_YHE010_3_Slot1-1_1_5095', 'P3856_YHE211_5_Slot1-1_1_5108',
       'P3856_YHE010_5_Slot1-1_1_5097', 'P3856_YHE114_7_Slot1-1_1_5121',
       'P3856_YHE114_6_Slot1-1_1_5120', 'P3856_YHE211_3_Slot1-1_1_5106',
       'P3856_YHE114_5_Slot1-1_1_5119', 'P3856_YHE114_9_Slot1-1_1_5123',
       'P3856_YHE211_7_Slot1-1_1_5110', 'P3856_YHE010_1_Slot1-1_1_5093',
       'P3856_YHE114_8_Slot1-1_1_5122', 'P3856_YHE010_9_Slot1-1_1_5101',
       'P3856_YHE114_2_Slot1-1_1_5116', 'P3856_YHE114_1_Slot1-1_1_5115',
       'P3856_YHE114_4_Slot1-1_1_5118', 'P3856_YHE211_4_Slot1-1_1_5107',
       'P3856_YHE211_6_Slot1-1_1_5109', 'P3856_YHE010_7_Slot1-1_1_5099',
       'P3856_YHE211_2_Slot1-1_1_5105', 'P3856_YHE010_10_Slot1-1_1_5102',
       'P3856_YHE211_8_Slot1-1_1_5111', 'P3856_Y

In [10]:
# restrict to the run and the RT range we're interested in
run_allpeptides_df = allpeptides_df[(allpeptides_df['Raw file'] == run_name) & (allpeptides_df.rt_in_seconds >= RT_LOWER) & (allpeptides_df.rt_in_seconds <= RT_UPPER)]


In [11]:
def collate_spectra_for_feature(ms1_d, ms2_df):
    # append the monoisotopic and the ms2 fragments to the list for MGF creation
    pairs_df = ms2_df[['mz', 'intensity']].copy().sort_values(by=['mz'], ascending=True)
    spectrum = {}
    spectrum["m/z array"] = pairs_df.mz.values
    spectrum["intensity array"] = pairs_df.intensity.values
    params = {}
    params["TITLE"] = "RawFile: {} Index: {} Charge: {} FeatureIntensity: {} RtApex: {}".format(ms1_d['raw_file'], ms1_d['mq_index'], ms1_d['charge'], ms1_d['intensity'], round(ms1_d['rt_apex'],2))
    params["INSTRUMENT"] = "ESI-QUAD-TOF"
    params["PEPMASS"] = "{} {}".format(round(ms1_d['monoisotopic_mz'],6), ms1_d['intensity'])
    params["CHARGE"] = "{}+".format(ms1_d['charge'])
    params["RTINSECONDS"] = "{}".format(round(ms1_d['rt_apex'],2))
    params["SCANS"] = "{}".format(int(ms1_d['rt_apex']))
    spectrum["params"] = params
    return spectrum

In [12]:
# build a list of indexes from the APL files
ms2_peaks = []
apl_indexes = []
for file in glob.glob("{}/*.apl".format(APL_DIR)):
    with open(file, 'r') as f:
        for line in f:
            line = line.rstrip()
            if len(line) > 0:
                if line.startswith("header="):
                    line_a = line.split(' ')
                    mq_index = int(line_a[3])
                    raw_file = line_a[1]
                if line[0].isdigit():
                    line_a = line.split('\t')
                    mz = float(line_a[0])
                    intensity = round(float(line_a[1]))
                    ms2_peaks.append((mz, intensity))
                if line.startswith("peaklist end"):
                    apl_indexes.append((mq_index, ms2_peaks.copy(), raw_file))
                    del ms2_peaks[:]
                    mq_index = 0
apl_indexes_df = pd.DataFrame(apl_indexes, columns=['mq_index','ms2_peaks','raw_file'])
apl_indexes_df.sort_values(by=['mq_index'], ascending=True, inplace=True)

In [13]:
for group_name,group_df in run_allpeptides_df.groupby('Raw file'):
    mgf_spectra = []
    visualisation_l = []
    mgf_file_name = '{}/{}-dwm.mgf'.format(BASE_MAXQUANT_DIR, group_name)
    df_file_name = '{}/{}-dwm.pkl'.format(BASE_MAXQUANT_DIR, group_name)
    file_apl_indexes_df = apl_indexes_df[(apl_indexes_df['raw_file'] == group_name)]
    for idx,row in group_df.iterrows():
        mq_index = row.msms_scan_number
        ms1_d = {'monoisotopic_mass':row.Mass, 
                 'charge':row.charge_state, 
                 'monoisotopic_mz':row.mz, 
                 'intensity':int(row.intensity), 
                 'scan_apex':row.scan, 
                 'rt_apex':row.rt_in_seconds,
                 'raw_file':row['Raw file'],
                 'mq_index':mq_index}
        df = file_apl_indexes_df[(file_apl_indexes_df.mq_index == mq_index)]
        ms2_peaks_df = pd.DataFrame(df.iloc[0].ms2_peaks, columns=['mz','intensity'])
        # construct the visualisation DF
        visualisation_l.append({**ms1_d, 'ms2_peaks':ms2_peaks_df.to_dict('records')})
        # construct the MGF
        feature_spectra = collate_spectra_for_feature(ms1_d, ms2_peaks_df)
        mgf_spectra.append(feature_spectra)
    # generate the MGF for all the features
    print("generating MGF: {}".format(mgf_file_name))
    if os.path.isfile(mgf_file_name):
        os.remove(mgf_file_name)
    f = mgf.write(output=mgf_file_name, spectra=mgf_spectra)
    # save the visualisation DF
    print("saving DF: {}".format(df_file_name))
    vis_df = pd.DataFrame(visualisation_l)
    vis_df.to_pickle(df_file_name)


generating MGF: /home/daryl/MQ-analysis-of-P3856/combined/P3856_YHE211_1_Slot1-1_1_5104-dwm.mgf
saving DF: /home/daryl/MQ-analysis-of-P3856/combined/P3856_YHE211_1_Slot1-1_1_5104-dwm.pkl
